In [7]:
!unzip tatrach-autoformer-main.zip

Archive:  tatrach-autoformer-main.zip
   creating: tatrach-autoformer-main/
  inflating: tatrach-autoformer-main/.gitignore  
   creating: tatrach-autoformer-main/dataset/
  inflating: tatrach-autoformer-main/dataset/Data_TaTrach.csv  
   creating: tatrach-autoformer-main/dataset/ETT-small/
  inflating: tatrach-autoformer-main/dataset/ETT-small/ETTh1.csv  
   creating: tatrach-autoformer-main/data_provider/
  inflating: tatrach-autoformer-main/data_provider/data_factory.py  
  inflating: tatrach-autoformer-main/data_provider/data_loader.py  
 extracting: tatrach-autoformer-main/data_provider/__init__.py  
  inflating: tatrach-autoformer-main/Dockerfile  
  inflating: tatrach-autoformer-main/environment.yml  
   creating: tatrach-autoformer-main/exp/
  inflating: tatrach-autoformer-main/exp/exp_basic.py  
  inflating: tatrach-autoformer-main/exp/exp_main.py  
 extracting: tatrach-autoformer-main/exp/__init__.py  
   creating: tatrach-autoformer-main/layers/
  inflating: tatrach-autoform

In [10]:
!pip install reformer-pytorch

In [11]:
import sys
sys.path.append("tatrach-autoformer-main")
import argparse
import os
import torch
from exp.exp_main import Exp_Main#exp stands for experiments
import random
import numpy as np
from utils.tools import dotdict

In [17]:
def config_param(num_units,batch_size,learning_rate,dropout_rate):
    args = dotdict()
    args.target = 'OT'
    args.des = 'test'
    args.dropout = dropout_rate
    args.num_workers = 10
    args.gpu = 0
    args.lradj = 'type1'
    args.devices = '0'
    args.use_gpu = True
    args.use_multi_gpu = False
    # if args.use_gpu and args.use_multi_gpu: #是否使用多卡的判断
    #     args.dvices = args.devices.replace(' ', '')
    #     device_ids = args.devices.split(',')
    #     args.device_ids = [int(id_) for id_ in device_ids]
    #     args.gpu = args.device_ids[0]
    args.freq = 'h'
    args.checkpoints = './checkpoints/'
    args.bucket_size = 4
    args.n_hashes = 4
    args.seq_len = 24
    args.label_len = 24
    args.pred_len = 24
    args.e_layers = 2
    args.d_layers = 1
    args.n_heads = 8
    args.factor = 1
    args.d_model = num_units
    args.des = 'Exp'
    args.itr = 1
    args.d_ff = 2048
    args.moving_avg = 25
    args.distil = True
    args.output_attention = False
    args.patience= 3
    args.learning_rate = learning_rate
    args.batch_size = batch_size
    args.embed = 'timeF'
    args.activation = 'gelu'
    args.use_amp = False
    args.loss = 'mse'

    args.train_epochs = 1
    args.is_training = True
    args.enc_in = 1
    args.dec_in = 1
    args.c_out = 1
    args.target = "q64"
    args.root_path = './dataset'
    args.data_path ='mucnuoc_gio_preprocess1.csv'
    args.model_id='q64'
    args.model = 'Autoformer'
    #args.model = 'Informer'
    args.data = 'custom'
    args.features = 'S'

    print('Args in experiment:')
    print(args)

    Exp = Exp_Main
    return args

In [20]:
def trainning(args):
    Exp = Exp_Main
    if args.is_training:
      for ii in range(args.itr):
          # setting record of experiments
          setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
              args.model_id,
              args.model,
              args.data,
              args.features,
              args.seq_len,
              args.label_len,
              args.pred_len,
              args.d_model,
              args.n_heads,
              args.e_layers,
              args.d_layers,
              args.d_ff,
              args.factor,
              args.embed,
              args.distil,
              args.des, ii)

          exp = Exp(args)  # set experiments
          print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
          exp.train(setting)

          print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
          mae, rmse, r2=exp.test(setting)

          if args.do_predict:
              print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
              exp.predict(setting, True)
          torch.cuda.empty_cache()
          return mae, rmse, r2
    else:
      ii = 0
      setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(args.model_id,
                                                                                                    args.model,
                                                                                                    args.data,
                                                                                                    args.features,
                                                                                                    args.seq_len,
                                                                                                    args.label_len,
                                                                                                    args.pred_len,
                                                                                                    args.d_model,
                                                                                                    args.n_heads,
                                                                                                    args.e_layers,
                                                                                                    args.d_layers,
                                                                                                    args.d_ff,
                                                                                                    args.factor,
                                                                                                    args.embed,
                                                                                                    args.distil,
                                                                                                    args.des, ii)

      exp = Exp(args)  # set experiments
      print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
      exp.test(setting, test=1)
      torch.cuda.empty_cache()

In [21]:
from sklearn.model_selection import ParameterGrid

# Danh sách các tham số để thử nghiệm
param_grid = {
    "num_units": [32, 64],  # Số nơ-ron trong LSTM
    "batch_size": [32,64],  # Kích thước batch
    "learning_rate": [0.01,0.001],  # Learning rate
    "dropout_rate": [0.2, 0.5],
}

# Lưu kết quả của từng bộ tham số
results = []

# Grid Search
for params in ParameterGrid(param_grid):
    print(f"Đang huấn luyện với tham số: {params}")

    # Xây dựng mô hình

    args=config_param(params["num_units"],params["batch_size"],params["learning_rate"],params["dropout_rate"])

    # Cấu hình optimizer
    mae, rmse, r2=trainning(args)
    # Lưu kết quả
    results.append((params, rmse,r2))

# Tìm bộ tham số tốt nhất
best_params = sorted(results, key=lambda x: x[1])[0]
print("Best Parameters:", best_params)

# Tìm bộ tham số tốt nhất
best_params = sorted(results, key=lambda x: x[2])[0]
print("Best Parameters:", best_params)

Đang huấn luyện với tham số: {'batch_size': 32, 'dropout_rate': 0.2, 'learning_rate': 0.01, 'num_units': 32}
Args in experiment:
{'target': 'q64', 'des': 'Exp', 'dropout': 0.2, 'num_workers': 10, 'gpu': 0, 'lradj': 'type1', 'devices': '0', 'use_gpu': True, 'use_multi_gpu': False, 'freq': 'h', 'checkpoints': './checkpoints/', 'bucket_size': 4, 'n_hashes': 4, 'seq_len': 24, 'label_len': 24, 'pred_len': 24, 'e_layers': 2, 'd_layers': 1, 'n_heads': 8, 'factor': 1, 'd_model': 32, 'itr': 1, 'd_ff': 2048, 'moving_avg': 25, 'distil': True, 'output_attention': False, 'patience': 3, 'learning_rate': 0.01, 'batch_size': 32, 'embed': 'timeF', 'activation': 'gelu', 'use_amp': False, 'loss': 'mse', 'train_epochs': 1, 'is_training': True, 'enc_in': 1, 'dec_in': 1, 'c_out': 1, 'root_path': './dataset', 'data_path': 'mucnuoc_gio_preprocess1.csv', 'model_id': 'q64', 'model': 'Autoformer', 'data': 'custom', 'features': 'S'}
Use GPU: cuda:0
>>>>>>>start training : q64_Autoformer_custom_ftS_sl24_ll24_pl24_

In [15]:
os.chdir("tatrach-autoformer-main")

FileNotFoundError: [Errno 2] No such file or directory: 'tatrach-autoformer-main'

In [16]:
import os

# In ra thư mục hiện tại
print("Current working directory:", os.getcwd())

# Liệt kê toàn bộ file/thư mục trong thư mục hiện tại
print("Files here:", os.listdir("."))


Current working directory: /content/tatrach-autoformer-main
Files here: ['exp', 'run.py', 'utils', 'layers', 'predict.ipynb', 'pic', 'environment.yml', 'dataset', '.gitignore', 'tatrach.rar', 'Dockerfile', 'README.md', 'tatrach.ipynb', 'models', 'LICENSE', 'Makefile', 'data_provider', 'requirements.txt', 'scripts']


In [ ]:
with open('exp/exp_main.py', 'r') as f:
    print(f.read())

import logging
logging.basicConfig(format='%(asctime)s,%(msecs)03d %(levelname)-8s [%(filename)s:%(lineno)d] %(message)s',
    datefmt='%Y-%m-%d:%H:%M:%S',
    level=logging.INFO)

from data_provider.data_factory import data_provider
from exp.exp_basic import Exp_Basic
from models import Informer, Autoformer, Transformer, Reformer
from utils.tools import EarlyStopping, adjust_learning_rate, visual
from utils.metrics import metric

import numpy as np
import torch
import torch.nn as nn
from torch import optim

import os
import time

import warnings
import matplotlib.pyplot as plt
import numpy as np

warnings.filterwarnings('ignore')


class Exp_Main(Exp_Basic):
    def __init__(self, args):
        super(Exp_Main, self).__init__(args)

    def _build_model(self):
        model_dict = {
            'Autoformer': Autoformer,
            'Transformer': Transformer,
            'Informer': Informer,
            'Reformer': Reformer,
        }
        model = model_dict[self.args.model].Model